# Finding Genes

Now that we have run RepeatMasker on our assembled genome, we are going to identify coding genes. To do this, we can
generally take a few different approaches:

1. Using RNA-seq data that we generated for our organism, which we can align to the genome, and then use it to identify
genes.

2. Using another similar organism for which we already have information on what genes should "look like". For example,
we can use already identified human genes to identify genes in a chimpanzee genome.

Today, we are going to use the first approach. In this section we are going to use RNA-seq data to identify genes in four basic steps:

1. Align RNA-seq data to our assembly.
2. Identify putative introns in these alignments.
3. Train a model that tells Augustus what a gene in our assembly looks like.
4. Identify genes with Augustus.

## Aligning RNA-seq data

For our input RNA-seq data we are going to use sequencing generated by a _P. falciparum_ project performed in part at the
Wellcome Sanger Institute in Cambridge, UK. We are going to use this data to help us build a model of what a gene in our
 assembled genome should look like. This data comes from the following paper:

Lia Chappell, Philipp Ross, Lindsey Orchard, Timothy J. Russell, Thomas D. Otto, Matthew Berriman, Julian C. Rayner, and Manuel Llinás. _**Refining the transcriptome of the human malaria parasite Plasmodium falciparum using amplification-free RNA-seq**_. BMC Genomics, 2020.

To generate the data used by this module, we accessed the sequencing data generated by the above paper on the
[European Nucleotide Archive](https://www.ebi.ac.uk/ena/browser/home) under the accession `ERX2287076`. This accession represents
paired-end Illumina-based short-read sequencing data generated for a single _P. falciparum_ isolate. We have already
filtered the reads to those which should align to the assembly that you generated as part of the assembly module.

Take a look at the data in your terminal:

In [ ]:
zcat Pfalc_chr5.1.fq.gz | head -n 12

This command should show you the first 3 reads in this file.

You should have already learned how to align RNA-seq data as part of the RNA-seq module. We are going to use the same
approach but for our assembly.

First, we have to index our assembly we made yesterday. This command makes it suitable for RNA-seq alignment with `hisat2`:

In [ ]:
hisat2-build PB.masked.fasta PB.masked.fasta.idx

Now we are going to align our reads to our reference genome:

In [ ]:
hisat2 --max-intronlen 10000 -x PB.masked.fasta.idx -1 Pfalc_chr5.1.fq.gz -2 Pfalc_chr5.2.fq.gz > Pfalc.sam

Now, convert the alignment to bam:

In [ ]:
samtools view -Sbo Pfalc.bam Pfalc.sam

Our gene finding approach that we will perform later requires an extra processing step. This tool will filter out
alignments that align to our assembly more than once:

In [ ]:
filterBam --uniq --in Pfalc.bam --out Pfalc.ssf.bam --paired --pairwiseAlignment

The `--paired` and `--pairwiseAlignment` tags tell `filterBam` that we are using paired-end data.

Now we sort and index the file as we did in the RNA-seq module:

In [ ]:
samtools sort -o Pfalc.sorted.ssf.bam Pfalc.ssf.bam

samtools index Pfalc.sorted.ssf.bam

We will take a look at these reads in IGV later to see how well our alignment worked.

We are now ready to start gene discovery.

## Converting RNA-seq data to intron information

We are now going to start using the Augustus tool. Augustus is available on github:

https://github.com/Gaius-Augustus/Augustus

Augustus was originally published as a free web-tool in the following paper:

  Mario Stanke and Burkhard Morgenstern. _**AUGUSTUS: a web server for gene prediction in eukaryotes that allows user-defined constraints**_. Nucleic Acids Research (2005).

To find genes, Augustus needs information on what a gene "looks like" in the organism that you have assembled. In
Eukaryotic genomes (such as _P. falciparum_), the most complicated part of gene discovery typically is identifying
introns/splice sites. Augustus thus uses the RNA-seq data we generated above to identify patterns in the DNA sequence
which are indicative of splicing.

This practical follows several parts of a tutorial developed by the Augustus authors which was published as part of the
following paper:

Katharina J. Hoff and Mario Stanke. _**Predicting Genes in Single Genomes with AUGUSTUS**_. Current Protocols in Bioinformatics (2018).

A free preprint of this publication, which is very similar to the published version, is available using
[this link](https://math-inf.uni-greifswald.de/storages/uni-greifswald/fakultaet/mnf/mathinf/stanke/augustus_wrp.pdf).

This workflow also makes use of several scripts from the tool BRAKER, also written by the Augustus authors. We have
copied the scripts required for this tutorial from BRAKER's [github repository](https://github.com/Gaius-Augustus/BRAKER)
for you to use today and placed them in the `~/course_data/annotation/scripts/` folder.

**Please note:** The method presented here is a simplification of the approach that you would need to take to accurately
 predict genes in a new genome and is presented here as a foundation on which to build. We have omitted several steps
 for the sake of computing resources and time. See the above Augustus tutorial for further steps that can be taken to
 more accurately predict genes in genome assemblies. There are other tools and resources that can be used
 to annotate both eukaryotic and prokaryotic genomes.

First, we need to find all the introns in our aligned RNA-sequencing data and then filter out low-quality introns:

In [ ]:
bam2hints --intronsonly --in=Pfalc.sorted.ssf.bam --out=Pfalc.introns.gff

filterIntronsFindStrand.pl PB.masked.fasta Pfalc.introns.gff --score > Pfalc.introns.f.gff

Next, we would normally use the ["GeneMark-ES"](http://exon.gatech.edu/GeneMark/gmes_instructions.html) software to
identify genes using our intron predictions. Unfortunately, the licence of GeneMark-ES prevents us from putting this
software on a virtual machine. As such, we have pre-run "GeneMark-ES" on the data in this module and provided the
necessary output for you already in the file `genemark.gtf`.

To generate this file we used the following command (Do not run this command!):

In [ ]:
gmes_petap.pl --verbose --sequence=PB.masked.fasta --ET=Pfalc.introns.f.gff --soft_mask 1000

While we cannot demonstrate how this code runs today, you can go to the
[GeneMark-ES website](http://exon.gatech.edu/GeneMark/gmes_instructions.html) and download a copy for your own personal
use after this course.

Let's take a look at the output of GeneMark-ES and see what it contains:

In [ ]:
head -n 31 genemark.gtf

This command should print predictions for the first two genes identified by GeneMark-ES, fittingly named "1_g" and "2_g".

**Questions:**

1. How many exons does the gene "1_g" have?
2. Can you think of a simple LINUX command to figure out how many genes GeneMark-ES identified?
3. How many genes did GeneMark find?

Next, we are going to use various commands from [BRAKER](https://github.com/Gaius-Augustus/BRAKER) to generate a list of "high quality" genes using the list of introns (`Pfalc.introns.f.gff`) and gene predictions from GeneMark-ES (`genemark.gtf`) that you made above:

In [ ]:
filterGenemark.pl genemark.gtf Pfalc.introns.f.gff

This command will generate two files: `genemark.f.good.gtf` and `genemark.f.bad.gtf` that we will use later.

This command should also give output like (actual output may differ slightly):

    Number of cds hints is 0
    Average gene length: 2614
    Average number of introns: 1.89261744966443
    Good gene rate: 0.422818791946309
    Number of genes: 300
    Number of complete genes: 298
    Number of good genes: 126
    Number of one-exon-genes: 104
    Number of bad genes: 174
    Good intron rate: 0.200570201421801
    One exon gene rate (of good genes): 0.349206349206349
    One exon gene rate (of all genes): 0.346666666666667

This output is telling us that, out of the total number of genes predicted by GeneMark-ES (300), only 126 look like
real genes according to Augustus.

Next, we are going to use scripts provided as part of Augustus to further filter our predicted genes down to a set of reliable hits. Run the following command:

In [ ]:
computeFlankingRegion.pl genemark.f.good.gtf

This command is simply calculating the total length of genes from our predictions above. It should generate an output similar to:

    Total length gene length (including introns): 220281. Number of genes: 321.
    Average Length: 686.233644859813

    The flanking_DNA value is: 343 (the Minimum of 10 000 and 343)

This is telling us that our predicted genes are, on average, about 670 basepairs long. We now need to use the value
provided by this script, or approximately half the mean gene length (343) in our following commands:

In [ ]:
gff2gbSmallDNA.pl genemark.gtf PB.masked.fasta 343 tmp.gb 2> /dev/null

filterGenesIn_mRNAname.pl genemark.f.good.gtf tmp.gb > bonafide.gb

This will generate our list of likely true genes (`bonafide.gb`) so that we can get to identifying actual genes in our
assembly. This file is in "genbank" format, the same format used by NCBI's GenBank to store sequence data about genes.
You can go to GenBank with the following link: https://www.ncbi.nlm.nih.gov/genbank/

**Questions:**

1. What is the part of the command `2> /dev/null` actually doing?
2. How many genes are in our final set of possible genes (`bonafide.gb`)?

We are now ready to train Augustus to actually find genes. If these commands did not work, we have placed a backup of
`bonafide.gb` in the `annotation_backups/` folder.

## Training Augustus for Gene Finding

We are now going to use our set of likely genes to "train" augustus on what a gene in our _P. falciparum_ assembly
looks like. This approach uses a test-training method, where we split our genes into two pieces and "train" augustus on
one half, and "test" on the other.

This approach allows us to remove statistical circularity. In other words, we can see how good our predictor is since
we don't use our testing set to train the data.

We have already created some files in the `species/` directory for you to use for this exercise. But we first need to
tell augustus that this directory is where we want our training data to go:

In [ ]:
export AUGUSTUS_CONFIG_PATH=/home/manager/course_data/annotation/data/

This is known in linux as setting an "environment variable". This allows programs (like Augustus) to know certain
pieces of information when they run. You can see that the environment variable is set properly by typing:

In [ ]:
echo $AUGUSTUS_CONFIG_PATH

This should print the same path that we set above.

We can now tell Augustus that we are going to train a new model, and Augustus will automatically place default files
in the directory `species/mygenome/`:

In [ ]:
new_species.pl --species=mygenome

etraining --species=mygenome bonafide.gb

You can put any name for `--species`, but for today we will use "mygenome". The first command sets up a working directory, and the second does an initial round of training on our genes. After running this command run the following:

In [ ]:
ls species/mygenome/

You should see the following output:

    mygenome_exon_probs.pbl    mygenome_intron_probs.pbl  mygenome_metapars.cgp.cfg
    mygenome_parameters.cfg    mygenome_igenic_probs.pbl  mygenome_metapars.cfg
    mygenome_metapars.utr.cfg  mygenome_weightmatrix.txt

These files contain information Augustus needs to identify genes. We are now going to identify better parameters using our test/training approach. First, we need to make a test and training set like we described above. In this case, we are going to simply split our data in half. But how many genes is that? To find out how many genes we have do:

In [ ]:
grep -c LOCUS bonafide.gb

This should give "126" (if it doesn't don't worry, just change the following accordingly). So, if we are going to split
50/50 to form train/test sets, we need to put 63 genes in both our training and test set (126 / 2 = 63). The
following command will do this for us:

In [ ]:
randomSplit.pl bonafide.gb 63

This will create two files: `bonafide.gb.train` and `bonafide.gb.test`.

Now, lets run training on our training set:

In [ ]:
etraining --species=mygenome bonafide.gb.train &> etrain.out

What new information did this teach us? Most importantly, it gave us information on how often different stop
codons are used. We can get this information from the "tail" of the output file with the following command:

In [ ]:
tail -n 6 etrain.out

This command simply prints the last 6 lines of this file, and you should see something like:

**Note:** Your numbers for the below _will be different_. This is because the output of the `randomSplit.pl` command is...
random!

    tag:    8 (0.127)
    taa:   44 (0.698)
    tga:   11 (0.175)
    end *EXON*
    Storing parameters to file...
    Writing exon model parameters [1] to file
    /home/manager/course_data/annotation/data/species/mygenome/mygenome_exon_probs.pbl.

This command takes care of most of the training, but we need to manually adjust a few parameters on our own.
We are going to use a text editor known as `gedit` to change the training parameters according to the above data.
Open the file we need to change:

In [ ]:
gedit species/mygenome/mygenome_parameters.cfg

Now find the lines by scrolling through the file our using "find" (ctrl+f) that look like:

    /Constant/amberprob                   0.33   # Prob(stop codon = tag), if 0 tag is assumed to ...
    /Constant/ochreprob                   0.33   # Prob(stop codon = taa), if 0 taa is assumed to ...
    /Constant/opalprob                    0.34   # Prob(stop codon = tga), if 0 tga is assumed to ...

And change them to match the output of our command above, like:

    /Constant/amberprob                   0.127   # Prob(stop codon = tag), if 0 tag is assumed to ...
    /Constant/ochreprob                   0.698   # Prob(stop codon = taa), if 0 taa is assumed to ...
    /Constant/opalprob                    0.175   # Prob(stop codon = tga), if 0 tga is assumed to ...

Make sure you *save* the file before exiting - by using either the "Save" button at the top-right or by hitting "ctrl+s" on your keyboard.

Finally, we are going to run Augustus on our test set and see how good our prediction is!

In [ ]:
augustus --species=mygenome bonafide.gb.test > test.out

So, how did we do? The last 41 lines of the file `test.out` tell us exactly how well we did:

In [ ]:
tail -n 41 test.out

This file provides 4 different tables, 1 for each of:

1. Nucleotide-level prediction - are individual bases predicted to be part of genes, found within genes?
2. Exon-level prediction - are individual exons predicted properly?
3. Transcript-level prediction - are entire transcripts predicted properly?
4. UTR-based prediction - We didn't train our approach for UTRs, ignore this table

For each, we get different statistics, but let's focus on sensitivity and specificity. These two values are commonly
used in test-training approaches and in machine learning like the one we have used here. For our exon level data, we
should have a table similar to this (the actual numbers will be different):

![](images/gene_1.png)

Now what do the values in this table mean in plain English:

* Sensitivity - How did we do in identifying real exons?
    * Out of the total number of actual true exons (159), we found 110 total true events:
        * 122/159 = 76.7%
* Specificity - How many exons did we identify that are not real exons?
    * Out of the total number of predictions (144), we found 34 additional events that are not actually exons:
        * 1 - (29 / 151) = 80.8%

Overall, this is a pretty good start for identifying exons. You may have noticed in one of the other tables that our
ability to predict entire transcripts is not nearly as good. We will visually evalute the results at the end of this
section to see why that may be.

**Question:** What do you think a limitation of using just 1 chromosome to train our gene finder is?

## Identifying Genes

Let's quickly review what we have done:

1. Aligned RNA-seq data to our assembly
2. Identified putative introns in these alignments
3. Trained a model that tells Augustus what a gene in our assembly looks like.

So, all that is left is to:

4. Identify genes

Now to actually run Augustus on our assembly:

In [ ]:
augustus --species=mygenome PB.masked.fasta > PB.contigs.gff

This command will likely take a minute or two depending on the computer you are using.

We are also going to run Augustus using their default parameters, trained for _P. falciparum_, to compare against:

In [ ]:
augustus --species=pfalciparum PB.masked.fasta > PB.contigs.default.gff

Each of these commands will generate a single gff format file. The ENSEMBL website has more information on the
format of [gff files](https://www.ensembl.org/info/website/upload/gff.html).

**Questions:**

1. Can you figure out how many genes each approach found?
2. We identified protein coding genes, but can you think of any other types of annotations we could find with Augustus?

> _hint: look back at the introduction to this practical._

If these commands didn't work or took too long to run, we have placed backups at `annotation_backups/PB.contigs*.gff`

## Checking our Annotations in IGV

Now that we have some genes, lets use the [Integrative Genomics Viewer](https://software.broadinstitute.org/software/igv/)
to explore our results. You should have used IGV as part of several of the modules of this course, including RNA-seq
and CHIP-seq. We will be using a similar approach here.

Go ahead and open IGV:

In [ ]:
igv &

The `&` symbol tells IGV to open in another window on your virtual machine's desktop, but allows you to continue using
your terminal. We are going to load your assembly as a genome in IGV.

Now, load your assembly, RNA-seq alignments, and gene data into IGV:

1. **Go to _"Genomes -> Load Genome From File..."_. Select "PB.masked.fasta" and click _"Open"_.**
2. **Go to _"File -> Load From File"_. Select "Pfalc.sorted.ssf.bam" and click _"Open"_.**
3. **Go to _"File -> Load From File"_. Select "PB.contigs.gff" and click _"Open"_.**
4. **Go to _"File -> Load From File"_. Select "PB.contigs.default.gff" and click _"Open"_.**

If you cannot find your files, make sure to navigate to the folder we are working in:

Go to **course_data/ --> annotation/ --> data/**

You should now have your assembly loaded into IGV, a set of RNA-seq reads labeled "Pfalc.sorted.bam Coverage" and
"Pfalc.sorted.bam" and two gene annotations. Make sure you "_right click_" on the gene annotation and click _"Squished"_
so that you can see exon-intron information. If you "Zoom in", your window should look similar to:

![](images/IGV_1.png)

Now, navigate to the top of the window and go to the coordinates:

    tig00000001:77,000-80,500

**Questions:**

1. How many exons does this gene have?
2. Do you think there are any issues in the gene structure?
3. How do the predictions for your model versus the default model compare?

Now go to the coordinates:

    tig00000001:123,400-126,800

**Question:** What are both predictions missing, and why do you think that is?

> _hint: look back at the introduction to this practical._

Finally, go to the coordinates:

    tig00000001:165,000-171,500

If you click on the gene structure (the one with exons and introns) of the gene in IGV, you should see a window like
the following:

![](images/IGV_2.png)

The "name" of this gene in the window is g54, which simply means that it is the 54th gene that Augustus identified.
Note that the name of your gene may be slightly different - please adjust the following commands accordingly.
We can then go back to our terminal and find this gene in our .gff file using a simple command like grep:

In [ ]:
grep -A 30 "start gene g54" PB.contigs.gff

* `-A` tells us how many lines "after" we match g54 we should report.

Make sure to modify your command if the gene is not named "g54". Note that this command will not work for every gene
that we identified exactly as written as they will have a different length (modifiable with the `-A` parameter) and a
different name.

**Questions:**

1. How many exons does this gene have?
2. How many introns?
3. How can you extract the same information for another gene by modifying the above command? Report the command and result here.

Feel free to explore the data in IGV before you continue to the final module,
[Comparative Genomics](comparative_genomics.ipynb).